In [ ]:
import pandas as pd
from rouge_score import rouge_scorer
import re
import six 

df = pd.read_csv('new_4o_ehr_review.csv')
df2 = pd.read_csv('35_ehr_review.csv')
df3 = pd.read_csv('o3_ehr_review.csv')
df4 = pd.read_csv('llama3_ehr.csv')

In [4]:


import re

def process_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    tokens = text.split()
    return tokens

def lcs(X, Y):
    m, n = len(X), len(Y)
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(m):
        for j in range(n):
            if X[i] == Y[j]:
                dp[i+1][j+1] = dp[i][j] + 1
            else:
                dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
    return dp[m][n]

def rouge_l_recall(pred, ref):
    pred_tokens = process_text(pred)
    ref_tokens = process_text(ref)
    if not ref_tokens:
        return 0.0
    lcs_len = lcs(pred_tokens, ref_tokens)
    return lcs_len / len(ref_tokens)

# Example usage:
generated_text = "61 y.o. female with a history of recurrent SBO s/p 07/26/20, severe PCM with GT feeding, pancreatic insufficiency, MCA aneurysm s/p clip/shunt, seizure disorder on 4AEDS, HTN, IDDM, COPD, and previous polysubstance abuse presenting with shortness of breath."
reference_text = "***** ***** is a ***** y.o. old female, with history of COPD, aspiration, CVA, Gtube, T2DM, seizure disorder, ***** as code 3 for respiratory distress"

score = rouge_l_recall(generated_text, reference_text)
print(f"ROUGE-L recall: {score:.4f}")



ROUGE-L recall: 0.3333


In [5]:
from collections import Counter

def get_ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

def rouge_n_recall(pred_text, ref_text, n=1):
    pred_tokens = pred_text.split()
    ref_tokens = ref_text.split()
    
    pred_ngrams = Counter(get_ngrams(pred_tokens, n))
    ref_ngrams = Counter(get_ngrams(ref_tokens, n))
    
    # Count overlap
    overlap = 0
    for ng in ref_ngrams:
        overlap += min(ref_ngrams[ng], pred_ngrams.get(ng, 0))
    
    total_ref_ngrams = sum(ref_ngrams.values())
    
    if total_ref_ngrams == 0:
        return 0.0
    return overlap / total_ref_ngrams

# Example usage:
generated_text = "61 y.o. female with a history of recurrent SBO s/p 07/26/20, severe PCM with GT feeding, pancreatic insufficiency, MCA aneurysm s/p clip/shunt, seizure disorder on 4AEDS, HTN, IDDM, COPD, and previous polysubstance abuse presenting with shortness of breath."
reference_text = "***** ***** is a ***** y.o. old female, with history of COPD, aspiration, CVA, Gtube, T2DM, seizure disorder, ***** as code 3 for respiratory distress"

rouge_1 = rouge_n_recall(generated_text, reference_text, n=1)
rouge_2 = rouge_n_recall(generated_text, reference_text, n=2)

print(f"ROUGE-1 recall: {rouge_1:.4f}")
print(f"ROUGE-2 recall: {rouge_2:.4f}")


ROUGE-1 recall: 0.2800
ROUGE-2 recall: 0.0417


In [ ]:
import pandas as pd
from evaluate import load

# Example DataFrame
df = pd.DataFrame({
    'generated_text': [
        "61 y.o. female with a history of recurrent SBO s/p 07/26/20, severe PCM with GT feeding, pancreatic insufficiency, MCA aneurysm s/p clip/shunt, seizure disorder on 4AEDS, HTN, IDDM, COPD, and previous polysubstance abuse presenting with shortness of breath."
    ],
    'reference_text': [ 
        "***** ***** is a ***** y.o. old female, with history of COPD, aspiration, CVA, Gtube, T2DM, seizure disorder, ***** as code 3 for respiratory distress"
    ]
})

# Optionally remove placeholder asterisks
def clean(text):
    return text.replace("*****", "").strip()

# Load ROUGE metric
rouge = load("rouge")

# Function to calculate ROUGE-L score
def compute_rouge_l(pred, ref):
    result = rouge.compute(predictions=[pred], references=[ref], use_aggregator=False)
    return result["rougeL"][0]

# Clean text (optional)
df["clean_pred"] = df["generated_text"].apply(clean)
df["clean_ref"] = df["reference_text"].apply(clean)

# Apply ROUGE-L row-wise
df["rougeL"] = df.apply(lambda row: compute_rouge_l(row["clean_pred"], row["clean_ref"]), axis=1)

# Output
print(df[["generated_text", "reference_text", "rougeL"]])


In [17]:
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

In [38]:
df["Generated_Summary"].isna().sum()

0

In [19]:
df2 = df2.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])
df2

,patientdurablekey,encounterkey,ArrivalDateKey,DepartureDateKeyValue,DepartureDateKey,DispositionDateKeyValue,primarychiefcomplaintname,primaryeddiagnosisname,sex,birthdate,...,Echo_Text,Imaging_Text,Consult_Text,ECG_Text,ED_Provider_Notes_Text,One_Sentence_Extracted,note_count,acuitylevel,eddisposition,Generated_Summary
0,DD644159F63417,D97E95EF247E4C,20230710,2023-07-11,20230711,2023-07-11,SHORTNESS OF BREATH,Hypoxia,Female,1961-08-23,...,NaN,CT ANGIO BRAIN/NECK FOR STROKE (INC 3D POST-PR...,"***** *****. *****, RN 04/16/2022 1:12 PM...",IMPRESSION: Borderline ECG Sinus tachycardi...,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED At...,"***** ***** is a ***** y.o. old female, with h...",7,Immediate,Admit,61-year-old female with history of recurrent S...
1,D76477E56C0B53,D70EAC2389B921,20221202,2022-12-04,20221204,2022-12-03,FALL,Fall from ground level,Female,1970-12-12,...,\n\nPatient Name: ***** ***** Patient I...,XR CHEST 2 VIEWS PA AND LATERAL 06/08/2017 3...,"***** ***** *****, MD 12/04/2022 4:22 PM""...",IMPRESSION: Normal sinus rhythm Possible La...,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED ...,"***** ***** is a ***** y.o. old female, who ha...",8,Urgent,Admit,51-year-old female with h/o active crack and I...
2,D76477E56C0B53,D0D09A7E783F07,20240106,2024-01-08,20240108,2024-01-06,MOUTH LESIONS,AIDS (acquired immune deficiency syndrome) (CM...,Female,1970-12-12,...,\n\nPatient Name: ***** ***** Patient I...,"XR CHEST 1 VIEW ***** 01/08/2023 4:20 PM""\n\...","***** ***** *****, MD 12/04/2022 4:22 PM""...",IMPRESSION: Normal sinus rhythm Possible La...,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED At...,"***** ***** is a ***** y.o. old female, with A...",8,Less Urgent,Admit,53-year-old female with h/o active crack and I...
3,D8F7D48FAEB29D,D486E6DF6FE817,20230205,2023-02-06,20230206,2023-02-06,HEMATURIA,Pyelonephritis,Female,1996-10-14,...,"University of *****, ...","XR ANKLE 3 VIEWS, LEFT 11/23/2020 5:21 PM ...",NaN,NaN,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED ...,***** ***** ***** ***** is a ***** y.o. old fe...,6,Urgent,Discharge,"26-year-old female, G1P0 with NSVD, postpartum..."
4,D8F7D48FAEB29D,DA51A78587E4AC,20230427,2023-04-28,20230428,2023-04-28,VAGINAL BLEEDING,PID (acute pelvic inflammatory disease),Female,1996-10-14,...,"University of *****, ...","XR ANKLE 3 VIEWS, LEFT 11/23/2020 5:21 PM ...",NaN,IMPRESSION: Normal sinus rhythm with sinus a...,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED At...,***** ***** ***** ***** is a ***** y.o. old fe...,7,Urgent,Discharge,"26-year-old female, G1P0 with NSVD on 06/12/20..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3979,D747CBD26BF2E5,D7B93BB4215531,20240202,2024-02-03,20240203,2024-02-03,ABDOMINAL PAIN,Bilateral flank pain,Female,2002-10-26,...,"University of *****, ...",CT ABDOMEN/PELVIS WITH AND WITHOUT CONTRAST ...,"***** *****, MD 09/24/2018 5:48 PM""\n\n\n...","Normal sinus rhythm""\n\nPrecordial lead mispla...",EMERGENCY DEPARTMENT PHYSICIAN NOTE ED At...,"***** is a ***** y.o. old female, with lupus c...",8,NaN,NaN,21-year-old female with a history of discharge...
3980,D8425274DB07BA,D88975CFC960BD,20220620,2022-06-21,20220621,2022-06-20,ABDOMINAL PAIN,"Ulcerative colitis with complication, unspecif...",Female,2002-02-09,...,NaN,XR ABDOMEN COMPLETE 09/05/2020 5:15 AM IN...,"***** ***** 07/29/2018 6:43 PM""\n\nIBD (u...","Borderline ECG""\n\nSinus tachycardia""\n\nWhen ...",EMERGENCY DEPARTMENT PHYSICIAN NOTE ED ...,"***** ***** is a ***** y.o. old female, with P...",7,NaN,NaN,20 y.o. female with history of ulcerative coli...
3981,D747CBD26BF2E5,D0B1EF59F6B2DF,20220720,2022-07-21,20220721,2022-07-20,OTHER,Complicated UTI (urinary tract infection),Female,2002-10-26,...,"University of *****, ...",CT ABDOMEN/PELVIS WITH AND WITHOUT CONTRAST ...,"***** *****, MD 09/24/2018 5:48 PM""\n\n\n...","Normal sinus rhythm""\n\nPrecordial lead mispla...",EMERGENCY DEPARTMENT PHYSICIAN NOTE ED ...,"***** is a ***** y.o. old female, with a PMH w...",8,NaN,NaN,

In [39]:
df2["Generated_Summary"].isna().sum()

17

In [20]:
df3 = df3.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])
df3

,patientdurablekey,encounterkey,ArrivalDateKey,DepartureDateKeyValue,DepartureDateKey,DispositionDateKeyValue,primarychiefcomplaintname,primaryeddiagnosisname,sex,birthdate,...,Echo_Text,Imaging_Text,Consult_Text,ECG_Text,ED_Provider_Notes_Text,One_Sentence_Extracted,note_count,acuitylevel,eddisposition,Generated_Summary
0,DD644159F63417,D97E95EF247E4C,20230710,2023-07-11,20230711,2023-07-11,SHORTNESS OF BREATH,Hypoxia,Female,1961-08-23,...,NaN,CT ANGIO BRAIN/NECK FOR STROKE (INC 3D POST-PR...,"***** *****. *****, RN 04/16/2022 1:12 PM...",IMPRESSION: Borderline ECG Sinus tachycardi...,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED At...,"***** ***** is a ***** y.o. old female, with h...",7,Immediate,Admit,61 y.o. female with h/o recurrent SBO s/p 07/2...
1,D76477E56C0B53,D70EAC2389B921,20221202,2022-12-04,20221204,2022-12-03,FALL,Fall from ground level,Female,1970-12-12,...,\n\nPatient Name: ***** ***** Patient I...,XR CHEST 2 VIEWS PA AND LATERAL 06/08/2017 3...,"***** ***** *****, MD 12/04/2022 4:22 PM""...",IMPRESSION: Normal sinus rhythm Possible La...,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED ...,"***** ***** is a ***** y.o. old female, who ha...",8,Urgent,Admit,51 y/o female with past history of AIDS (CD4 8...
2,D76477E56C0B53,D0D09A7E783F07,20240106,2024-01-08,20240108,2024-01-06,MOUTH LESIONS,AIDS (acquired immune deficiency syndrome) (CM...,Female,1970-12-12,...,\n\nPatient Name: ***** ***** Patient I...,"XR CHEST 1 VIEW ***** 01/08/2023 4:20 PM""\n\...","***** ***** *****, MD 12/04/2022 4:22 PM""...",IMPRESSION: Normal sinus rhythm Possible La...,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED At...,"***** ***** is a ***** y.o. old female, with A...",8,Less Urgent,Admit,"53 y.o. f, with h/o AIDS (advanced HIV with CD..."
3,D8F7D48FAEB29D,D486E6DF6FE817,20230205,2023-02-06,20230206,2023-02-06,HEMATURIA,Pyelonephritis,Female,1996-10-14,...,"University of *****, ...","XR ANKLE 3 VIEWS, LEFT 11/23/2020 5:21 PM ...",NaN,NaN,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED ...,***** ***** ***** ***** is a ***** y.o. old fe...,6,Urgent,Discharge,26 y.o. female with hx of normal labor with ep...
4,D8F7D48FAEB29D,DA51A78587E4AC,20230427,2023-04-28,20230428,2023-04-28,VAGINAL BLEEDING,PID (acute pelvic inflammatory disease),Female,1996-10-14,...,"University of *****, ...","XR ANKLE 3 VIEWS, LEFT 11/23/2020 5:21 PM ...",NaN,IMPRESSION: Normal sinus rhythm with sinus a...,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED At...,***** ***** ***** ***** is a ***** y.o. old fe...,7,Urgent,Discharge,26 y.o. female with h/o normal spontaneous vag...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3979,D747CBD26BF2E5,D7B93BB4215531,20240202,2024-02-03,20240203,2024-02-03,ABDOMINAL PAIN,Bilateral flank pain,Female,2002-10-26,...,"University of *****, ...",CT ABDOMEN/PELVIS WITH AND WITHOUT CONTRAST ...,"***** *****, MD 09/24/2018 5:48 PM""\n\n\n...","Normal sinus rhythm""\n\nPrecordial lead mispla...",EMERGENCY DEPARTMENT PHYSICIAN NOTE ED At...,"***** is a ***** y.o. old female, with lupus c...",8,NaN,NaN,21 y.o. female with a past ED visit on 08/04/2...
3980,D8425274DB07BA,D88975CFC960BD,20220620,2022-06-21,20220621,2022-06-20,ABDOMINAL PAIN,"Ulcerative colitis with complication, unspecif...",Female,2002-02-09,...,NaN,XR ABDOMEN COMPLETE 09/05/2020 5:15 AM IN...,"***** ***** 07/29/2018 6:43 PM""\n\nIBD (u...","Borderline ECG""\n\nSinus tachycardia""\n\nWhen ...",EMERGENCY DEPARTMENT PHYSICIAN NOTE ED ...,"***** ***** is a ***** y.o. old female, with P...",7,NaN,NaN,20 y.o. female with h/o ulcerative colitis wit...
3981,D747CBD26BF2E5,D0B1EF59F6B2DF,20220720,2022-07-21,20220721,2022-07-20,OTHER,Complicated UTI (urinary tract infection),Female,2002-10-26,...,"University of *****, ...",CT ABDOMEN/PELVIS WITH AND WITHOUT CONTRAST ...,"***** *****, MD 09/24/2018 5:48 PM""\n\n\n...","Normal sinus rhythm""\n\nPrecordial lead mispla...",EMERGENCY DEPARTMENT PHYSICIAN NOTE ED ...,"***** is a ***** y.o. old female, with a PMH w...",8,NaN,NaN,19

In [40]:
df3["Generated_Summary"].isna().sum()

0

In [23]:
df4 = df4.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

KeyError: "['Unnamed: 0' 'Unnamed: 0.1' 'Unnamed: 0.1.1'] not found in axis"

In [41]:
df4["Assistant_Content"].isna().sum()

0

In [25]:
df4["Assistant_Content"][4]

'\n\n26 y.o. female, with h/o G1P1, previous normal labor and delivery, epidural, 2nd degree perineal laceration, postpartum course: normal, no pending tests or complications, and currently presenting with vaginal bleeding.'

In [30]:
df4

,Unnamed: 0.2,patientdurablekey,encounterkey,ArrivalDateKey,DepartureDateKeyValue,DepartureDateKey,DispositionDateKeyValue,primarychiefcomplaintname,primaryeddiagnosisname,sex,...,Imaging_Text,Consult_Text,ECG_Text,ED_Provider_Notes_Text,One_Sentence_Extracted,note_count,acuitylevel,eddisposition,Assistant_Content,Processed_One_Sentence_Extracted
0,0,DD644159F63417,D97E95EF247E4C,20230710,2023-07-11,20230711,2023-07-11,SHORTNESS OF BREATH,Hypoxia,Female,...,CT ANGIO BRAIN/NECK FOR STROKE (INC 3D POST-PR...,"***** *****. *****, RN 04/16/2022 1:12 PM...",IMPRESSION: Borderline ECG Sinus tachycardi...,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED At...,"***** ***** is a ***** y.o. old female, with h...",7,Immediate,Admit,"80 y.o. female, with h/o HFpEF (EF 55-60% 05/2...",is a y o old female with history of copd aspi...
1,1,D76477E56C0B53,D70EAC2389B921,20221202,2022-12-04,20221204,2022-12-03,FALL,Fall from ground level,Female,...,XR CHEST 2 VIEWS PA AND LATERAL 06/08/2017 3...,"***** ***** *****, MD 12/04/2022 4:22 PM""...",IMPRESSION: Normal sinus rhythm Possible La...,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED ...,"***** ***** is a ***** y.o. old female, who ha...",8,Urgent,Admit,51 y/o female with h/o PSA (active crack and I...,is a y o old female who has a past medical hi...
2,2,D76477E56C0B53,D0D09A7E783F07,20240106,2024-01-08,20240108,2024-01-06,MOUTH LESIONS,AIDS (acquired immune deficiency syndrome) (CM...,Female,...,"XR CHEST 1 VIEW ***** 01/08/2023 4:20 PM""\n\...","***** ***** *****, MD 12/04/2022 4:22 PM""...",IMPRESSION: Normal sinus rhythm Possible La...,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED At...,"***** ***** is a ***** y.o. old female, with A...",8,Less Urgent,Admit,"""53 y/o female with h/o HIV (10/2015 CD4 246k,...",is a y o old female with aids p w cough and o...
3,3,D8F7D48FAEB29D,D486E6DF6FE817,20230205,2023-02-06,20230206,2023-02-06,HEMATURIA,Pyelonephritis,Female,...,"XR ANKLE 3 VIEWS, LEFT 11/23/2020 5:21 PM ...",NaN,NaN,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED ...,***** ***** ***** ***** is a ***** y.o. old fe...,6,Urgent,Discharge,"26 y.o. female with h/o hypertension, HLD, bip...",is a y o old female with hx of anxiety asthma...
4,4,D8F7D48FAEB29D,DA51A78587E4AC,20230427,2023-04-28,20230428,2023-04-28,VAGINAL BLEEDING,PID (acute pelvic inflammatory disease),Female,...,"XR ANKLE 3 VIEWS, LEFT 11/23/2020 5:21 PM ...",NaN,IMPRESSION: Normal sinus rhythm with sinus a...,EMERGENCY DEPARTMENT PHYSICIAN NOTE ED At...,***** ***** ***** ***** is a ***** y.o. old fe...,7,Urgent,Discharge,"26 y.o. female, with h/o G1P1, previous normal...",is a y o old female with pmh of gastritis and...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3979,3979,D747CBD26BF2E5,D7B93BB4215531,20240202,2024-02-03,20240203,2024-02-03,ABDOMINAL PAIN,Bilateral flank pain,Female,...,CT ABDOMEN/PELVIS WITH AND WITHOUT CONTRAST ...,"***** *****, MD 09/24/2018 5:48 PM""\n\n\n...","Normal sinus rhythm""\n\nPrecordial lead mispla...",EMERGENCY DEPARTMENT PHYSICIAN NOTE ED At...,"***** is a ***** y.o. old female, with lupus c...",8,NaN,NaN,Here's a concise one-liner summary for the pat...,is a y o old female with lupus cystitis s p r...
3980,3980,D8425274DB07BA,D88975CFC960BD,20220620,2022-06-21,20220621,2022-06-20,ABDOMINAL PAIN,"Ulcerative colitis with complication, unspecif...",Female,...,XR ABDOMEN COMPLETE 09/05/2020 5:15 AM IN...,"***** ***** 07/29/2018 6:43 PM""\n\nIBD (u...","Borderline ECG""\n\nSinus tachycardia""\n\nWhen ...",EMERGENCY DEPARTMENT PHYSICIAN NOTE ED ...,"***** ***** is a ***** y.o. old female, with P...",7,NaN,NaN,"20 y.o. female, with a history of ulcerative c...",is a y o old female with pmh ulcerative colit...
3981,3981,D747CBD26BF2E5,D0B1EF59F6B2DF,20220720,2022-07-21,20220721,2022-07-20,OTHER,Complicated UTI (urinary tract infection),Female,...,CT ABDOMEN/PELVIS WITH AND WITHOUT CONTRAST ...,"***** *****, MD 09/24/2018 5:48 PM""\n\n\n...","Normal sinus rhythm""\n\nPrecordial lead mispl

In [26]:
def clean_medical_summary(text):
    # Remove newline characters
    cleaned = text.replace('\n', ' ').replace('\r', ' ')
    
    # Strip extra spaces
    cleaned = ' '.join(cleaned.split())
    
    return cleaned
df4["Assistant_Content"] = df4["Assistant_Content"].apply(clean_medical_summary)

In [29]:
df4["Assistant_Content"][4]

'26 y.o. female, with h/o G1P1, previous normal labor and delivery, epidural, 2nd degree perineal laceration, postpartum course: normal, no pending tests or complications, and currently presenting with vaginal bleeding.'

In [31]:

# Define a function to process text (for example, to make it lowercase and clean non-alpha-numeric characters)
def process_text(text):
    # Ensure the text is a string, handle NaN or invalid data by converting it to a string or empty string
    if not isinstance(text, str):
        text = str(text)  # Convert to string if it's not a string (e.g., for float or NaN values)
    
    # Convert everything to lowercase.
    text = text.lower()
    
    # Replace any non-alpha-numeric characters with spaces.
    text = re.sub(r"[^a-z0-9]+", " ", six.ensure_str(text))
    
    return text

# Example usage: Process text columns in your DataFrame
df['Processed_One_Sentence_Extracted'] = df['One_Sentence_Extracted'].apply(process_text)
df['Processed_Generated_Summary'] = df['Generated_Summary'].apply(process_text)

# Similarly, you can process text in df2 or any other DataFrame
df2['Processed_One_Sentence_Extracted'] = df2['One_Sentence_Extracted'].apply(process_text)
df2['Processed_Generated_Summary'] = df2['Generated_Summary'].apply(process_text)

# Similarly, you can process text in df2 or any other DataFrame
df3['Processed_One_Sentence_Extracted'] = df3['One_Sentence_Extracted'].apply(process_text)
df3['Processed_Generated_Summary'] = df3['Generated_Summary'].apply(process_text)


# Similarly, you can process text in df2 or any other DataFrame
df4['Processed_One_Sentence_Extracted'] = df4['Processed_One_Sentence_Extracted'].apply(process_text)
df4['Processed_Generated_Summary'] = df4['Assistant_Content'].apply(process_text)


In [32]:
len(df)

3984

In [33]:
len(df2)

3984

In [34]:
len(df3)

3984

In [35]:
len(df4)

3984

In [36]:

# Define function to compute ROUGE scores and F1 scores
def compute_rouge_scores(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    
    # Extract ROUGE scores
    rouge1_precision = scores['rouge1'].precision
    rouge1_recall = scores['rouge1'].recall
    rouge1_fmeasure = scores['rouge1'].fmeasure
    
    rouge2_precision = scores['rouge2'].precision
    rouge2_recall = scores['rouge2'].recall
    rouge2_fmeasure = scores['rouge2'].fmeasure
    
    rougeL_precision = scores['rougeL'].precision
    rougeL_recall = scores['rougeL'].recall
    rougeL_fmeasure = scores['rougeL'].fmeasure
    
    # Calculate F1 for each ROUGE score
    f1_rouge1 = 2 * (rouge1_precision * rouge1_recall) / (rouge1_precision + rouge1_recall) if (rouge1_precision + rouge1_recall) != 0 else 0
    f1_rouge2 = 2 * (rouge2_precision * rouge2_recall) / (rouge2_precision + rouge2_recall) if (rouge2_precision + rouge2_recall) != 0 else 0
    f1_rougeL = 2 * (rougeL_precision * rougeL_recall) / (rougeL_precision + rougeL_recall) if (rougeL_precision + rougeL_recall) != 0 else 0
    
    # Return ROUGE F1 scores for the DataFrame
    return pd.Series([rouge1_fmeasure, rouge2_fmeasure, rougeL_fmeasure, f1_rouge1, f1_rouge2, f1_rougeL], 
                     index=['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-1-F1', 'ROUGE-2-F1', 'ROUGE-L-F1'])



# Compute ROUGE scores for df using processed columns
df[['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-1-F1', 'ROUGE-2-F1', 'ROUGE-L-F1']] = df.apply(
    lambda row: compute_rouge_scores(row['Processed_One_Sentence_Extracted'], row['Processed_Generated_Summary']),
    axis=1
)

# Compute ROUGE scores for df2 using processed columns
df2[['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-1-F1', 'ROUGE-2-F1', 'ROUGE-L-F1']] = df2.apply(
    lambda row: compute_rouge_scores(row['Processed_One_Sentence_Extracted'], row['Processed_Generated_Summary']),
    axis=1
)

# Compute ROUGE scores for df2 using processed columns
df3[['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-1-F1', 'ROUGE-2-F1', 'ROUGE-L-F1']] = df3.apply(
    lambda row: compute_rouge_scores(row['Processed_One_Sentence_Extracted'], row['Processed_Generated_Summary']),
    axis=1
)

# Compute ROUGE scores for df2 using processed columns
df4[['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-1-F1', 'ROUGE-2-F1', 'ROUGE-L-F1']] = df4.apply(
    lambda row: compute_rouge_scores(row['Processed_One_Sentence_Extracted'], row['Assistant_Content']),
    axis=1
)

# Compute the aggregated (mean) ROUGE scores and F1 scores for df
aggregated_rouge_f1_scores_df = df[['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-1-F1', 'ROUGE-2-F1', 'ROUGE-L-F1']].mean()

# Compute the aggregated (mean) ROUGE scores and F1 scores for df2
aggregated_rouge_f1_scores_df2 = df2[['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-1-F1', 'ROUGE-2-F1', 'ROUGE-L-F1']].mean()

# Compute the aggregated (mean) ROUGE scores and F1 scores for df2
aggregated_rouge_f1_scores_df3 = df3[['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-1-F1', 'ROUGE-2-F1', 'ROUGE-L-F1']].mean()

# Compute the aggregated (mean) ROUGE scores and F1 scores for df2
aggregated_rouge_f1_scores_df4 = df4[['ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-1-F1', 'ROUGE-2-F1', 'ROUGE-L-F1']].mean()

# Print the aggregated scores for df
print(f'Aggregated ROUGE and F1 Scores for GPT-4o:\n{aggregated_rouge_f1_scores_df}')

# Print the aggregated scores for df2
print(f'Aggregated ROUGE and F1 Scores for GPT-3.5:\n{aggregated_rouge_f1_scores_df2}')


# Print the aggregated scores for df2
print(f'Aggregated ROUGE and F1 Scores for O3-mini:\n{aggregated_rouge_f1_scores_df3}')


# Print the aggregated scores for df2
print(f'Aggregated ROUGE and F1 Scores for LLama3:\n{aggregated_rouge_f1_scores_df4}')


Aggregated ROUGE and F1 Scores for GPT-4o:
ROUGE-1       0.346749
ROUGE-2       0.126664
ROUGE-L       0.285771
ROUGE-1-F1    0.346749
ROUGE-2-F1    0.126664
ROUGE-L-F1    0.285771
dtype: float64
Aggregated ROUGE and F1 Scores for GPT-3.5:
ROUGE-1       0.309570
ROUGE-2       0.131968
ROUGE-L       0.262131
ROUGE-1-F1    0.309570
ROUGE-2-F1    0.131968
ROUGE-L-F1    0.262131
dtype: float64
Aggregated ROUGE and F1 Scores for O3-mini:
ROUGE-1       0.311885
ROUGE-2       0.120407
ROUGE-L       0.271613
ROUGE-1-F1    0.311885
ROUGE-2-F1    0.120407
ROUGE-L-F1    0.271613
dtype: float64
Aggregated ROUGE and F1 Scores for LLama3:
ROUGE-1       0.293899
ROUGE-2       0.110161
ROUGE-L       0.250804
ROUGE-1-F1    0.293899
ROUGE-2-F1    0.110161
ROUGE-L-F1    0.250804
dtype: float64
